In [3]:
from z3 import *

In [4]:
# 5x5 rooks problem
#R_1 is the rook on the first column, R_3 rook on third column etc
# R_1=2 means the rook on first column is on the second row etc
R=[Int('R_%i' % (i+1)) for i in range(5)]
print(R)
print("")

#rooks between 1 and 5 in value
valConstraint=[And(R[i]>0,R[i]<6) for i in range(5)]
print(valConstraint)
print("")

#rooks can't be on the same row
rowConstraint=[Distinct(R)]
print(rowConstraint)
print("")

s=Solver()
s.add(valConstraint)
s.add(rowConstraint)

print(s.check())
print(s.model())

[R_1, R_2, R_3, R_4, R_5]

[And(R_1 > 0, R_1 < 6), And(R_2 > 0, R_2 < 6), And(R_3 > 0, R_3 < 6), And(R_4 > 0, R_4 < 6), And(R_5 > 0, R_5 < 6)]

[Distinct(R_1, R_2, R_3, R_4, R_5)]

sat
[R_1 = 1, R_2 = 2, R_3 = 3, R_4 = 4, R_5 = 5]


In [5]:
#a,b values assignment

#a[0]=a1
a=[Int('a%i' % (i)) for i in range(1,11)]
print(a)
print("")
b=[Int('b%i' % (i)) for i in range(1,11)]
for i, v in enumerate(b):
     print(i, v)
print("")

#TEST visualize in python
# for i in range(1,11):
#     if a>b:
#         b=2*b
#         a=a-3
#     else:
#         a=2*a
#         b=b-5
s=Solver()

#TEST first is the initial value, then the subsequent values depend on the former ones
# s.add( If(a[0]>b[0], And(b[1]==2*b[0],a[1]==a[0]-3), And( a[1]==2*a[0], b[1]==b[0]-5 ) ) )

#looping the condition
for i in range(1,10):
    s.add( If(a[i-1]>b[i-1], And(b[i]==2*b[i-1],a[i]==a[i-1]-3), And( a[i]==2*a[i-1], b[i]==b[i-1]-5 ) ) )
#last values
s.add(a[9]==1000,b[9]==999)

print(s.check())
print(s.model())

[a1, a2, a3, a4, a5, a6, a7, a8, a9, a10]

0 b1
1 b2
2 b3
3 b4
4 b5
5 b6
6 b7
7 b8
8 b9
9 b10

sat
[a4 = 128,
 a2 = 32,
 b6 = 256,
 b5 = 128,
 b4 = 133,
 b2 = 143,
 a6 = 253,
 b1 = 148,
 a5 = 256,
 a7 = 506,
 a1 = 16,
 b3 = 138,
 b7 = 251,
 a8 = 503,
 a3 = 64,
 b8 = 502,
 b9 = 1004,
 a9 = 500,
 b10 = 999,
 a10 = 1000]


In [6]:
#3 wolf 3 rabbits
time=IntSort()
number=IntSort()
pos, (left, right) = EnumSort('pos', ['left', 'right'])
wolves= Function('wolves',time,pos,number)
rabbits=Function('rabbits',time,pos,number)

s=Solver()
#any animals at any time/position must be between 0 and 3
def minMaxAnimals(time,pos):
    return(
        And(  wolves(time,pos)>=0, wolves(time,pos)<4 , rabbits(time,pos)>=0, rabbits(time,pos)<4  )
     )
#if at any time/pos rabbits>wolves true, otherwise if at least 1rabbit 1wolf check if the number of wolves smaller than rabbits
def notEaten(time,pos):
    return(
        If(
            wolves(time,pos)<=rabbits(time,pos),
            True,
            If(
                And(wolves(time,pos)>0,rabbits(time,pos)>0),
                If(
                    wolves(time,pos)<=rabbits(time,pos),
                    True,
                    False
                ),
                True
            )
        )
    )
#going from 1 step to another step and checking the constraints
def  crossLeftRight(time1,time2):
    return ( And(
        minMaxAnimals(time1,left),
        minMaxAnimals(time1,right),
        minMaxAnimals(time2,left),
        minMaxAnimals(time2,right),

        notEaten(time1,left),
        notEaten(time1,right),
        notEaten(time2,left),
        notEaten(time2,right),
        
        Or(

            #wolf crosses alone
            And(
                wolves(time2,left)==wolves(time1,left)-1,
                wolves(time2,right)==wolves(time1,right)+1,
                rabbits(time2,left)==rabbits(time1,left),
                rabbits(time2,right)==rabbits(time1,right)
            ),
            #2wolves cross
            And(
                wolves(time2,left)==wolves(time1,left)-2,
                wolves(time2,right)==wolves(time1,right)+2,
                rabbits(time2,left)==rabbits(time1,left),
                rabbits(time2,right)==rabbits(time1,right)
            ),
            #1 wolf one rabbit cross
            And(
                wolves(time2,left)==wolves(time1,left)-1,
                wolves(time2,right)==wolves(time1,right)+1,
                rabbits(time2,left)==rabbits(time1,left)-1,
                rabbits(time2,right)==rabbits(time1,right)+1
            ),
            #1 rabbit cross
            And(
                wolves(time2,left)==wolves(time1,left),
                wolves(time2,right)==wolves(time1,right),
                rabbits(time2,left)==rabbits(time1,left)-1,
                rabbits(time2,right)==rabbits(time1,right)+1
            ),
            # 2 rabbits cross
            And(
                wolves(time2,left)==wolves(time1,left),
                wolves(time2,right)==wolves(time1,right),
                rabbits(time2,left)==rabbits(time1,left)-2,
                rabbits(time2,right)==rabbits(time1,right)+2
            )
        
        )
    )        
    )
#going from 1 step to another step and checking the constraints
def crossRightLeft(time1,time2):
    return(
        And(
            minMaxAnimals(time1,left),
            minMaxAnimals(time1,right),
             minMaxAnimals(time2,left),
             minMaxAnimals(time2,right),

            notEaten(time1,left),
            notEaten(time1,right),
            notEaten(time2,left),
            notEaten(time2,right),
            Or(
               #wolf crosses alone
               And(
                   wolves(time2,left)==wolves(time1,left)+1,
                   wolves(time2,right)==wolves(time1,right)-1,
                   rabbits(time2,left)==rabbits(time1,left),
                   rabbits(time2,right)==rabbits(time1,right)
               ),
               #2 wolves cross
               And(
                   wolves(time2,left)==wolves(time1,left)+2,
                   wolves(time2,right)==wolves(time1,right)-2,
                   rabbits(time2,left)==rabbits(time1,left),
                   rabbits(time2,right)==rabbits(time1,right)
               ),
               #1wolf 1rabbit cross
               And(
                   wolves(time2,left)==wolves(time1,left)+1,
                   wolves(time2,right)==wolves(time1,right)-1,
                   rabbits(time2,left)==rabbits(time1,left)+1,
                   rabbits(time2,right)==rabbits(time1,right)-1
               ),
               #1 rabbit cross
               And(
                   wolves(time2,left)==wolves(time1,left),
                   wolves(time2,right)==wolves(time1,right),
                   rabbits(time2,left)==rabbits(time1,left)+1,
                   rabbits(time2,right)==rabbits(time1,right)-1
               ),
               #2 rabbit cross
               And(
                   wolves(time2,left)==wolves(time1,left),
                   wolves(time2,right)==wolves(time1,right),
                   rabbits(time2,left)==rabbits(time1,left)+2,
                   rabbits(time2,right)==rabbits(time1,right)-2
               ),
            )         
        )
    )

#at time 0 all animals are on left side
s.add( wolves(0,left)==3 )
s.add( rabbits(0,left)==3 )
s.add( wolves(0,right)==0 )
s.add( rabbits(0,right)==0 )

#walk through the steps
s.add( crossLeftRight(0,1)  )
s.add( crossRightLeft(1,2)  )
s.add( crossLeftRight(2,3)  )
s.add( crossRightLeft(3,4)  )
s.add( crossLeftRight(4,5)  )
s.add( crossRightLeft(5,6)  )
s.add( crossLeftRight(6,7)  )
s.add( crossRightLeft(7,8)  )
s.add( crossLeftRight(8,9)  )
s.add( crossRightLeft(9,10)  )
s.add( crossLeftRight(10,11)  )


#at time 11 all animals are on right side
s.add( wolves(11,left)==0 )
s.add( rabbits(11,left)==0 )
s.add( wolves(11,right)==3 )
s.add( rabbits(11,right)==3 )

print(s.check())
print(s.model())
# print(s.sexpr())

# m = s.model()
# print ( m.evaluate(wolves(0,left)) )

sat
[rabbits = [(0, left) -> 3,
            (1, left) -> 2,
            (1, right) -> 1,
            (2, left) -> 3,
            (3, left) -> 3,
            (4, left) -> 3,
            (5, left) -> 1,
            (5, right) -> 2,
            (6, left) -> 2,
            (6, right) -> 1,
            (7, right) -> 3,
            (8, right) -> 3,
            (9, right) -> 3,
            (10, right) -> 3,
            (11, right) -> 3,
            else -> 0],
 wolves = [(0, left) -> 3,
           (0, right) -> 0,
           (1, right) -> 1,
           (2, right) -> 1,
           (3, left) -> 0,
           (3, right) -> 3,
           (4, left) -> 1,
           (5, left) -> 1,
           (6, right) -> 1,
           (7, right) -> 1,
           (8, left) -> 3,
           (8, right) -> 0,
           (9, left) -> 1,
           (10, right) -> 1,
           (11, left) -> 0,
           (11, right) -> 3,
           else -> 2]]


In [14]:
#bridge torch crossing
time=IntSort()
#time lapsed at time time
lapsed=Function('lapsed',time,IntSort())
#each person has a position at a certain time
alb=Function('alb',time,pos)
bob=Function('bob',time,pos)
cha=Function('cha',time,pos)
fre=Function('fre',time,pos)

# ------------RIGHT CROSSING------------------

#fred alone cross right
def freCrossedRight(t1,t2):
    return(
        If(
            And(
                fre(t1)==left,
                fre(t2)==right                
            ),
            And(
                lapsed(t2)==lapsed(t1)+1,
                cha(t2)==cha(t1),
                bob(t2)==bob(t1),
                alb(t2)==alb(t1)
            ),
            False
        )
    )

#cha crossed right(maybe fred as well)
def chaCrossedRight(t1,t2):
    return(
        If(
            And(
                cha(t1)==left,
                cha(t2)==right
            ),
            And(
                lapsed(t2)==lapsed(t1)+2,
                bob(t2)==bob(t1),
                alb(t2)==alb(t1),
                #fred can only cross if on the same side
                If(fre(t1)==cha(t1),True,fre(t1)==fre(t2))
            ),
            False
        )
    )

#bob crossed right(maybe with fred/charlie)
def bobCrossedRight(t1,t2):
    return(
        If(
            And(
                bob(t1)==left,
                bob(t2)==right
            ),
            And(
                lapsed(t2)==lapsed(t1)+5,
                alb(t2)==alb(t1),
                #if charlie pases with bob then fred doesnt and the other way around
                If(fre(t2)!=fre(t1),cha(t2)==cha(t1),True),
                If(cha(t1)!=cha(t2),fre(t1)==fre(t2),True)
            ),
            False
        )
    )

#alb crossed right(maybe with fred/charlie/bob)
def albCrossedRight(t1,t2):
    return(
        If(
            And(
                alb(t1)==left,
                alb(t2)==right
            ),
            And(
                lapsed(t2)==lapsed(t1)+10,
                #if albert pases with bob then fred/charlie doesnt and the other way around
                If( fre(t2)!=fre(t1),And( cha(t2)==cha(t1),bob(t2)==bob(t1)),True  ),
                If(cha(t1)!=cha(t2),And( fre(t1)==fre(t2),bob(t2)==bob(t1)),True ),
                If(bob(t1)!=bob(t2),And( cha(t2)==cha(t1),fre(t1)==fre(t2)),True )
            ),
            False
        )
    )

#check right crossing
def crossRight(t1,t2):
    return(
        Or(
            albCrossedRight(t1,t2),
            bobCrossedRight(t1,t2),
            chaCrossedRight(t1,t2),
            freCrossedRight(t1,t2)
        )
    )

# ------------LEFT CROSSING------------------

#fred alone cross left
def freCrossedLeft(t1,t2):
    return(
        If(
            And(
                fre(t1)==right,
                fre(t2)==left                
            ),
            And(
                lapsed(t2)==lapsed(t1)+1,
                cha(t2)==cha(t1),
                bob(t2)==bob(t1),
                alb(t2)==alb(t1)
            ),
            False
        )
    )

#cha crossed left(maybe fred as well)
def chaCrossedLeft(t1,t2):
    return(
        If(
            And(
                cha(t1)==right,
                cha(t2)==left
            ),
            And(
                lapsed(t2)==lapsed(t1)+2,
                bob(t2)==bob(t1),
                alb(t2)==alb(t1),
                #fred can only cross if on the same side
                If(fre(t1)==cha(t1),True,fre(t1)==fre(t2))
            ),
            False
        )
    )

#bob crossed left(maybe with fred/charlie)
def bobCrossedLeft(t1,t2):
    return(
        If(
            And(
                bob(t1)==right,
                bob(t2)==left
            ),
            And(
                lapsed(t2)==lapsed(t1)+5,
                alb(t2)==alb(t1),
                If(fre(t2)!=fre(t1),cha(t2)==cha(t1),True),
                If(cha(t1)!=cha(t2),fre(t1)==fre(t2),True)
            ),
            False
        )
    )

#alb crossed left(maybe with fred/charlie/bob)
def albCrossedLeft(t1,t2):
    return(
        If(
            And(
                alb(t1)==right,
                alb(t2)==left
            ),
            And(
                lapsed(t2)==lapsed(t1)+10,
                #if albert pases with bob then fred/charlie doesnt and the other way around
                If(fre(t2)!=fre(t1),And( cha(t2)==cha(t1),bob(t2)==bob(t1)),True ),
                If(cha(t1)!=cha(t2),And( fre(t1)==fre(t2),bob(t2)==bob(t1)),True ),
                If(bob(t1)!=bob(t2),And( cha(t2)==cha(t1),fre(t1)==fre(t2)),True )
            ),
            False
        )
    )

#check left crossing
def crossLeft(t1,t2):
    return(
        Or(
            albCrossedLeft(t1,t2),
            bobCrossedLeft(t1,t2),
            chaCrossedLeft(t1,t2),
            freCrossedLeft(t1,t2)
        )
    )


# s=Solver()
s=Optimize()
#there are 5 crossings needed to move all people from one side to the other
#all left
s.add(lapsed(0)==0)
s.add(alb(0)==left)
s.add(bob(0)==left)
s.add(cha(0)==left)
s.add(fre(0)==left)

s.add(crossRight(0,1))
s.add(crossLeft(1,2))
s.add(crossRight(2,3))
s.add(crossLeft(3,4))
s.add(crossRight(4,5))

#all right
s.add(alb(5)==right)
s.add(bob(5)==right)
s.add(cha(5)==right)
s.add(fre(5)==right)
#17 minutes it he minimum time to get an answer
# s.add(lapsed(5)<18)
s.minimize(lapsed(5))

print(s.check())
print(s.model())

sat
[alb = [3 -> right, 4 -> right, 5 -> right, else -> left],
 fre = [1 -> right, 5 -> right, else -> left],
 bob = [3 -> right, 4 -> right, 5 -> right, else -> left],
 lapsed = [1 -> 2,
           2 -> 3,
           3 -> 13,
           4 -> 15,
           5 -> 17,
           else -> 0],
 cha = [0 -> left, 4 -> left, else -> right]]
